In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=95076101fc307c89b51e94db0f38f52c30af273325805d0f43c4fab36969eea3
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").appName("Datamanipulation").getOrCreate()

In [4]:
spark

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd

In [7]:
# read our data - lives in a csv file

#df = spark.read.option("header","true").csv("/content/drive/MyDrive/Sample - EU Superstore.csv")
df = spark.read.csv("/content/drive/MyDrive/Sample - EU Superstore.csv", header=True, inferSchema=True)

In [8]:
df.show(20)

+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|  Aaron Smayling|  Corporate|        Leeds|             England|United Kingdom|  North|OFF-ST-10000988|Office Supplies|     Storage|Fellowes 

In [9]:
df.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         10000 non-null  int32  
 1   Order ID       10000 non-null  object 
 2   Order Date     10000 non-null  object 
 3   Ship Date      10000 non-null  object 
 4   Ship Mode      10000 non-null  object 
 5   Customer ID    10000 non-null  object 
 6   Customer Name  10000 non-null  object 
 7   Segment        10000 non-null  object 
 8   City           10000 non-null  object 
 9   State          10000 non-null  object 
 10  Country        10000 non-null  object 
 11  Region         10000 non-null  object 
 12  Product ID     10000 non-null  object 
 13  Category       10000 non-null  object 
 14  Sub-Category   10000 non-null  object 
 15  Product Name   10000 non-null  object 
 16  Sales          10000 non-null  float64
 17  Quantity       10000 non-null  int32  
 18  Discoun

In [10]:
df.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)



In [11]:
# how many rows of the EU Superstore dataset have the country being France
df.filter(df['Country'] == 'France').count()

2827

In [12]:
# of those, how many are profitable?
df.filter((df['Country'] == 'France') & 
          (df['Profit'] > 0)).count()


2329

In [13]:
# how any different discount brackets exist? what are they?
discount_brackets = df.select("Discount").distinct()
discount_brackets.count()

14

In [14]:
#what are they?
discount_brackets.show()

+--------+
|Discount|
+--------+
|     0.0|
|     0.2|
|     0.7|
|     0.1|
|    0.45|
|     0.6|
|     0.8|
|    0.35|
|     0.5|
|     0.4|
|    0.85|
|    0.15|
|    0.65|
|     0.3|
+--------+



In [15]:
# let's see the totl profit by discount bracket, make sure they are ordered by 
df.groupBy("Discount").agg({"Profit" : 'sum'}).orderBy("Discount").show()

+--------+-------------------+
|Discount|        sum(Profit)|
+--------+-------------------+
|     0.0| 383806.53000000026|
|     0.1|  126884.0309999999|
|    0.15| 24677.563499999975|
|     0.2| 2189.5499999999984|
|     0.3| -758.4209999999999|
|    0.35|          -9122.649|
|     0.4|-21346.427999999996|
|    0.45|         -1103.1915|
|     0.5|         -96632.115|
|     0.6|-20517.456000000002|
|    0.65| -6221.965499999999|
|     0.7|          -5496.765|
|     0.8|           -460.284|
|    0.85|          -3068.658|
+--------+-------------------+



In [16]:
# what is the value after which we should stop offering discount?
total_profit_by_discount = df.groupBy("Discount").agg({"Profit" : 'sum'}).withColumnRenamed("sum(Profit)", "total_profit")
total_profit_by_discount.filter(total_profit_by_discount.total_profit <= 0).orderBy("Discount").show()

+--------+-------------------+
|Discount|       total_profit|
+--------+-------------------+
|     0.3| -758.4209999999999|
|    0.35|          -9122.649|
|     0.4|-21346.427999999996|
|    0.45|         -1103.1915|
|     0.5|         -96632.115|
|     0.6|-20517.456000000002|
|    0.65| -6221.965499999999|
|     0.7|          -5496.765|
|     0.8|           -460.284|
|    0.85|          -3068.658|
+--------+-------------------+



We should stop offering more than 20 percent discount or 0.2 because after that we start to run into losses.

In [17]:
# who are the top 5 most profitable customers
df.groupBy("Customer Name").agg({"Profit" : 'sum'}).orderBy("sum(Profit)", ascending = False).limit(5).show()


+-----------------+------------------+
|    Customer Name|       sum(Profit)|
+-----------------+------------------+
|     Susan Pistek| 4974.512999999999|
|    Patrick Jones|3986.0039999999995|
|Patrick O'Donnell|          3778.197|
|    Ellis Ballard|           3459.66|
|  Mike Gockenbach|3144.4439999999995|
+-----------------+------------------+



In [18]:
# get all the rows belonging to those 5 customer names: hint, you may need the collect method - how many rows are they?
top_5_profitable = df.groupBy("Customer Name").agg({"Profit" : 'sum'}).orderBy("sum(Profit)", ascending = False).limit(5)
top_profitable_customers = top_5_profitable.collect()
top_profitable_customers_names = [row[0] for row in top_profitable_customers]
top_profitable_customers_rows = df.filter(df['Customer Name'].isin(top_profitable_customers_names))
top_profitable_customers_rows.show()

+------+---------------+----------+----------+--------------+-----------+-----------------+---------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+-------+--------+--------+-------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|    Customer Name|  Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|  Sales|Quantity|Discount| Profit|
+------+---------------+----------+----------+--------------+-----------+-----------------+---------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+-------+--------+--------+-------+
|     8|ES-2015-5113958|02/08/2015|07/08/2015|  Second Class|   EB-13840|    Ellis Ballard|Corporate|West Bromwich|             England|United Kingdom|  North|TEC-CO-10004325|     Technology|     Copiers|Canon Personal C

In [19]:
#how many rows are they?
top_profitable_customers_rows.count()

76

In [20]:
# create a new column which is the value of the sale were there not discount applied. Hint: orginal = sales/(1-d)

In [21]:
import pyspark.sql.functions as func
df = df.withColumn('No_discount_sales', func.round(df['Sales']/(1-df['Discount']),2))
df.show()

+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+-----------------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|No_discount_sales|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+-----------------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|  Aaron Smayling|  Corporate|        Leeds|             England|United Kingdom|  North|

In [22]:
# calculate the difference between sales and discount value
df.withColumn('sales_discount_difference' , func.round(df["No_discount_sales"] - df["Sales"])).select('No_discount_sales', 'Sales', 'sales_discount_difference').show()

+-----------------+--------+-------------------------+
|No_discount_sales|   Sales|sales_discount_difference|
+-----------------+--------+-------------------------+
|             79.2|    79.2|                      0.0|
|           388.92|  388.92|                      0.0|
|            35.19|   35.19|                      0.0|
|            50.94|   50.94|                      0.0|
|           307.44|  307.44|                      0.0|
|            122.4|   122.4|                      0.0|
|           413.82|  413.82|                      0.0|
|           428.22|  428.22|                      0.0|
|          3979.29| 3979.29|                      0.0|
|            43.56|   43.56|                      0.0|
|            25.26|   25.26|                      0.0|
|          2715.45|2443.905|                    272.0|
|            12.21|   12.21|                      0.0|
|          2549.76|2167.296|                    382.0|
|           153.45| 138.105|                     15.0|
|         

In [23]:
# how much money did we not gain due to the discounts - per discount bracket?
df = df.withColumn('sales_discount_difference' , func.round(df["No_discount_sales"] - df["Sales"]))
df.groupBy("Discount").sum("sales_discount_difference").orderBy("Discount").show()

+--------+------------------------------+
|Discount|sum(sales_discount_difference)|
+--------+------------------------------+
|     0.0|                           0.0|
|     0.1|                       84707.0|
|    0.15|                       45236.0|
|     0.2|                       10657.0|
|     0.3|                        2630.0|
|    0.35|                       29166.0|
|     0.4|                       46733.0|
|    0.45|                        2083.0|
|     0.5|                      183747.0|
|     0.6|                       39648.0|
|    0.65|                       12218.0|
|     0.7|                        8534.0|
|     0.8|                         636.0|
|    0.85|                        4516.0|
+--------+------------------------------+



In [24]:
# find the discount bracket which made us not gain the most (dynamically)
sum_loss = df.groupBy("Discount").sum("sales_discount_difference").orderBy("Discount").orderBy("sum(sales_discount_difference)", ascending = False)
most_loss = sum_loss.collect()[0][0]
most_loss

0.5

In [25]:
# what would have been the total profit if we removed all orders from that discount group? 
df_filtered = df.filter(df["Discount"] != most_loss)
total_profit = df_filtered.agg({"Profit" : "sum"}).withColumnRenamed("sum(Profit)", "sum_profit_after")
total_profit.show()

+-----------------+
| sum_profit_after|
+-----------------+
|469461.8565000003|
+-----------------+



In [26]:
#how much more (or less) profit is that?
total_profit_before = df.agg({"Profit" : "sum"}).withColumnRenamed("sum(Profit)", "sum_profit_before")
joined_total_profit = total_profit.join(total_profit_before)
joined_total_profit.show()

+-----------------+-----------------+
| sum_profit_after|sum_profit_before|
+-----------------+-----------------+
|469461.8565000003|372829.7415000005|
+-----------------+-----------------+



In [27]:
difference = total_profit["sum_profit_after"] - total_profit_before["sum_profit_before"]
difference = joined_total_profit.withColumn("Difference", (joined_total_profit["sum_profit_after"] - joined_total_profit["sum_profit_before"]))
difference.show()

+-----------------+-----------------+-----------------+
| sum_profit_after|sum_profit_before|       Difference|
+-----------------+-----------------+-----------------+
|469461.8565000003|372829.7415000005|96632.11499999976|
+-----------------+-----------------+-----------------+



In [28]:
# create a temporary table for our superstore table in sql
df.createOrReplaceTempView("eu_df")

In [29]:
# use an SQL query to count the number of rows
spark.sql("SELECT COUNT(*) FROM eu_df").show()

+--------+
|count(1)|
+--------+
|   10000|
+--------+



In [53]:
# Use an SQL query to calculate the profit ratio for each country: hint, ratio is sum(profit)/sum(sales)
spark.sql("SELECT Country, (SUM(Profit)/SUM(Sales)) AS Profit_ratio FROM eu_df GROUP BY Country ORDER BY Profit_ratio DESC").show()

+--------------+--------------------+
|       Country|        Profit_ratio|
+--------------+--------------------+
|   Switzerland|  0.2909201193350232|
|       Austria|  0.2641908775042505|
|        Norway|  0.2517747548521659|
|       Belgium| 0.23508766583987942|
|United Kingdom| 0.21170103540397134|
|         Spain| 0.18941580658358978|
|       Finland| 0.18864296633316185|
|       Germany| 0.17066792076621765|
|        France| 0.12693568221933804|
|         Italy| 0.06844355185424991|
|       Ireland|-0.44426677493909256|
|       Denmark| -0.4957190005664471|
|   Netherlands| -0.5298342790541865|
|        Sweden| -0.5745674280714466|
|      Portugal| -0.5761662270806188|
+--------------+--------------------+



In [57]:
profit_ratio_total_profit = spark.sql("SELECT country, SUM(profit) as total_profit, (SUM(Profit)/SUM(Sales)) AS Profit_ratio FROM eu_df GROUP BY country ORDER BY total_profit DESC")
profit_ratio_total_profit.show()


+--------------+-------------------+--------------------+
|       country|       total_profit|        Profit_ratio|
+--------------+-------------------+--------------------+
|United Kingdom| 111900.15000000001| 0.21170103540397134|
|        France| 109029.00299999975| 0.12693568221933804|
|       Germany| 107322.82049999991| 0.17066792076621765|
|         Spain|  54390.11999999999| 0.18941580658358978|
|       Austria|           21442.26|  0.2641908775042505|
|         Italy| 19828.757999999965| 0.06844355185424991|
|       Belgium|           11572.59| 0.23508766583987942|
|   Switzerland|  7237.470000000001|  0.2909201193350232|
|        Norway|            5167.77|  0.2517747548521659|
|       Finland|            3905.73| 0.18864296633316185|
|       Denmark|-4282.0470000000005| -0.4957190005664471|
|       Ireland| -7392.381000000003|-0.44426677493909256|
|      Portugal| -8703.059999999998| -0.5761662270806188|
|        Sweden|-17519.366999999987| -0.5745674280714466|
|   Netherland

In [56]:
# is the country with the largest profit ratio, the country with the largest profit?
largest_profit_and_profit_ratio = spark.sql("""SELECT country, SUM(profit) as total_profit, (SUM(profit)/SUM(sales)) AS profit_ratio 
                            FROM eu_df 
                            GROUP BY country 
                            HAVING SUM(profit) = (SELECT MAX(total_profit) FROM (SELECT country, SUM(profit) as total_profit FROM eu_df GROUP BY country)) 
                            AND (SUM(profit)/SUM(sales)) = (SELECT MAX(profit_ratio) FROM (SELECT country, (SUM(profit)/SUM(sales)) AS profit_ratio FROM eu_df GROUP BY country))""")

largest_profit_and_profit_ratio.show()

+-------+------------+------------+
|country|total_profit|profit_ratio|
+-------+------------+------------+
+-------+------------+------------+



From the empty table returned it means that the country with the largest ratio is not the country with the largest profit.
The country with the largest profit is the United Kingdom while the country with the largest profit ratio is Switzerland
